In [1]:
from pyspark.sql import SparkSession
from pyspark.sql import SQLContext
import pyspark.sql.functions as f
import pyspark

In [2]:
file_name = './data/News_Final.csv'
sc = pyspark.SparkContext(appName='hw2-1')
text_file = sc.textFile(file_name)
sqlContext=SQLContext(sc)
df=sqlContext.read.format('csv')\
                    .option("header", "true")\
                    .option("inferschema", "true")\
                    .option("mode", "DROPMALFORMED")\
                    .load(file_name)
df=df.withColumn('SentimentTitle', df['SentimentTitle'].cast('double'))
df=df.withColumn('SentimentHeadline', df['SentimentHeadline'].cast('double'))
df=df.withColumn('PublishDate', df['PublishDate'].cast('date'))
df.show()

# df.printSchema()

+------+--------------------+--------------------+--------------------+---------+-----------+-------------------+--------------------+--------+----------+--------+
|IDLink|               Title|            Headline|              Source|    Topic|PublishDate|     SentimentTitle|   SentimentHeadline|Facebook|GooglePlus|LinkedIn|
+------+--------------------+--------------------+--------------------+---------+-----------+-------------------+--------------------+--------+----------+--------+
| 99248|Obama Lays Wreath...|Obama Lays Wreath...|           USA TODAY|    obama| 2002-04-02|                0.0| -0.0533001790889026|      -1|        -1|      -1|
| 10423|A Look at the Hea...|Tim Haywood, inve...|           Bloomberg|  economy| 2008-09-20|  0.208333333333333|  -0.156385810542806|      -1|        -1|      -1|
| 18828|Nouriel Roubini: ...|Nouriel Roubini, ...|           Bloomberg|  economy| 2012-01-28| -0.425210032135381|   0.139754248593737|      -1|        -1|      -1|
| 27788|Finland 

In [3]:
df.select("Title","Topic","Headline").show()


+--------------------+---------+--------------------+
|               Title|    Topic|            Headline|
+--------------------+---------+--------------------+
|Obama Lays Wreath...|    obama|Obama Lays Wreath...|
|A Look at the Hea...|  economy|Tim Haywood, inve...|
|Nouriel Roubini: ...|  economy|Nouriel Roubini, ...|
|Finland GDP Expan...|  economy|Finland's economy...|
|Tourism, govt spe...|  economy|Tourism and publi...|
|Intellitec Soluti...|microsoft|Over 100 attendee...|
| Monday, 29 Feb 2016|palestine|RAMALLAH, Februar...|
|Obama, stars pay ...|    obama|First lady Michel...|
|Fire claims more ...|palestine|A Hancock County ...|
|Microsoft's new W...|microsoft|New Delhi, Feb.29...|
|Microsoft Project...|microsoft|Microsoft may hav...|
|Microsoft sneaks ...|microsoft|The platform batt...|
|Greek economy gro...|  economy|Greece's economy ...|
|Big data and the ...|  economy|Big data analytic...|
|HoloLens dev edit...|microsoft|Microsoft’s AR he...|
|Microsoft Word fo...|micros

In [4]:
data=df.where(df["Topic"]=="microsoft").select("Topic","Title","PublishDate","Headline")

In [5]:
columnName=['obama','economy','microsoft','palestine']
for name in columnName:
    print(name+"----------------Title")
    data=df.where(df["Topic"]==name).select("Topic","Title","PublishDate")
    data.withColumn('word', f.explode(f.split(f.col('Title'), ' ')))\
        .groupBy('PublishDate',"word")\
        .count()\
        .sort('count','PublishDate', ascending=False)\
        .write.format("csv").mode("overwrite").save("file:///home/bigdata/Documents/BD_HW/HW2/output/"+name+"_Title.csv")
    
#     data.select("Topic", "Title","PublishDate").write()\
#     .format("com.databricks.spark.csv")\
#     .option("header", "true")\
#     .option("codec", "org.apache.hadoop.io.compress.GzipCodec")\
#     .save("newcars.csv");
#     data.write.mode("overwrite").format("text").save("./test.txt")
#     out=open('output/SentimentScore.txt','w')
#     out.write('SentimentScore_sum\n')
#     print(SentimentScore_sum)
#     for i in range(len(SentimentScore_sum)):
#         out.write(str(SentimentScore_sum[i]) + '\n')
#     out.write('\nSentimentScore_average\n')
#     for i in range(len(SentimentScore_average)):
#         out.write(str(SentimentScore_average[i]) + '\n')
#     out.close()

In [6]:
for name in columnName:
    print(name+"----------------Headline")
    data=df.where(df["Topic"]==name).select("Topic","Title","PublishDate",'Headline')
    data.withColumn('word', f.explode(f.split(f.col('Headline'), ' ')))\
        .groupBy('PublishDate',"word")\
        .count()\
        .sort('count','PublishDate', ascending=False)\
        .write.format("csv").mode("overwrite").save("file:///home/bigdata/Documents/BD_HW/HW2/output/"+name+"_Headline.csv")
   

obama----------------Headline
economy----------------Headline
microsoft----------------Headline
palestine----------------Headline


In [7]:
sc.stop()